Problem 23 from the advent of code calendar 2022.

January 4, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import signal


np.set_printoptions(linewidth=250)

In [2]:
cdf = pd.read_csv('23.txt', header = None)
cdf.columns = ['dstring']

def process_string(s):
    r = []
    for i in s:
        if i == '.':
            r.append(0)
        elif i == '#':
            r.append(1)
        else:
            print("Weird character {}".format(i))
    return np.array(r)

In [3]:
a = cdf.dstring.apply(process_string).values
field = np.array(list(a))

In [4]:
###
# GLOBAL VARIABLES
###

#Add these to a location to increment 1
N = np.array([0,-1])
S = np.array([0,1])
E = np.array([1,0])
W = np.array([-1,0])

ORDERING = [N,S,W,E]

#NOTE:  These are adjusted to work with scipy's convolve2D which flips the grid around.
Sgrid = np.vstack([np.ones(3),np.zeros((2,3))])
Ngrid = np.vstack([np.zeros((2,3)),np.ones(3)])
Wgrid = np.hstack([np.zeros((3,2)),np.ones((3,1))])
Egrid = np.hstack([np.ones((3,1)),np.zeros((3,2))])
NEIGHBORS = np.ones((3,3))
NEIGHBORS[1,1] = 0

ORDERED_FILTERS = [Ngrid,Sgrid,Wgrid,Egrid,]

#Roll parameters:  Need these to shift the arrays appropriately
ROLL_PARAMETERS = [[-1,0],[1,0],[-1,1],[1,1]]
ANTIROLL_PARAMETERS = [[1,0],[-1,0],[1,1],[-1,1]]


In [5]:
###
# Helper functions for dealing with the field
###

def expand_field(f, border_width = 10):
    return np.pad(f,border_width)


def contract_field(f):
    """
    All hail stack overflow because np.trim_zeros is 1D only
    https://stackoverflow.com/questions/55917328/numpy-trim-zeros-in-2d-or-3d
    """
    nz = np.nonzero(f)  # Indices of all nonzero elements
    arr_trimmed = f[nz[0].min():nz[0].max()+1,
                  nz[1].min():nz[1].max()+1]
    return arr_trimmed


#Will want to contract the initial field and then expand it out by the number of iterations + filter size (i.e. 13)

In [6]:
###
# Filter functions
###

def get_filter_mask(grid,filter_grid):
    """
    Inputs:
        grid        a numpy array of 0s and positive numbers where each positive number corresponds to an elf 
                    at that location
        filter_grid a 3x3 numpy array which when convolved with area returns a positive number if the action
                    of the grid is *not* possible.  WARNING: Mess with the above grids in the globals at your \\
                    own peril.
    
    Returns an array the same size as the grid array where 1 denotes there is an elf that can do whatever 
    the filter grid is filtering for (i.e. supposed to stay, move to the north, etc.)
    """
    
    convo = signal.convolve2d(grid,filter_grid, mode = 'same')
    mask = np.ones_like(convo)
    mask[convo > 0] = 0
    return mask*grid







In [7]:
###
# Trying to perform a step
###

def move_elves(grid,inds,verbose = False):
    """
    Inputs:
        grid        a numpy array of 0s and 1s where each 1 corresponds to an elf 
                    at that location  #NOTE:  This function requires elves to be denoted by 1 for deduping to work.
        inds        some ordering of 0,1,2,3 which indicates the ordering of the directions to try
    """
    
    neighbors = get_filter_mask(grid,NEIGHBORS)
    left = grid - neighbors
    masks = {}
    proposed = neighbors.copy()
    for i in inds:
        #Get the elves who can go in this direction
        masks[i] = get_filter_mask(grid,ORDERED_FILTERS[i])
        #Only keep the elves who haven't already decided on a direction
        masks[i] = masks[i]*left
        #Remove all of the elves who have chosen this direction
        left = left - masks[i]
        #Shift the proposed mask and add it into proposed so that we can find duplicates (in the next loop we dedup)
        proposed = proposed + np.roll(masks[i],*ROLL_PARAMETERS[i])
    
    #Next we need to go through and dedup stuff:
    stationary = grid.copy()
    elves_moved = 0
    for i in inds:
        #First roll the mask appropriately
        rolled_mask = np.roll(masks[i],*ROLL_PARAMETERS[i])
        #Then turn all of the cases where the rolled masks coincides with a duplicate in proposed to 0
        rolled_mask[np.where(proposed > 1)] = 0
        #Unroll parameters so we don't loose elves
        masks[i] = np.roll(rolled_mask,*ANTIROLL_PARAMETERS[i])
        #And subtract out the elves who are actually going to move with this direction from the stationary tracker
        stationary = stationary - masks[i]
        #and now we add in the rolled masks to the stationary to move the elves
        stationary = stationary + rolled_mask
        elves_moved = elves_moved + rolled_mask.sum()
    if verbose:
        print("\t Elves moved is {}".format(elves_moved))
    return stationary
    

In [8]:
###
#
# And now that the step works, must iterate through the steps
# 
###

def run_field(grid,num_iters = 10, start_inds = np.array([0,1,2,3]),verbose = False):
    """
    THIS ASSUMES THAT THE GRID HAS BEEN PRECONDITIONED SO THAT ALL OF THE OUTSIDE OF THE GRID IS A LAYER OF 
    ZEROS AT LEAST num_iters WIDE!!!  Violate this assumption at your own risk.
    
    """
    
    inds = start_inds.copy()
    stae = grid.copy()
    for i in range(num_iters):
        if verbose:
            #print(inds)
            print ("Running round {}".format(i+1))
        stae = move_elves(stae,inds,verbose = verbose)
        inds = np.roll(inds,-1)
    return stae
    

def score_field(grid,num_iters = 10,verbose = False):
    """
    This function is hardcoded to the problem.
    """
    
    #pad the field
    fld = expand_field(grid, border_width=3+num_iters)
    if verbose:
        print("Field shape is now {} with {} elves".format(fld.shape, fld.sum()))
    fld = run_field(fld,num_iters = num_iters,start_inds = np.array([0,1,2,3]), verbose = verbose)
    fld = contract_field(fld)
    #Now we just need to compute the number of zeros
    score = (np.ones_like(fld) - fld).sum()
    return score


def run_field_until_static(grid, start_inds = np.array([0,1,2,3]),verbose = True):
    
    #Initialize the zero state
    inds = start_inds.copy()
    stae = grid.copy()
    stae = contract_field(stae)
    stae = expand_field(stae, border_width=5)
    rounds = 0
    
    #Create the post one state
    staen = move_elves(stae,inds)
    inds = np.roll(inds,-1)
    moved = (np.abs(staen - stae)).sum()/2.0
    rounds = rounds + 1
    while moved > 0:
        if verbose:
            print ("Round:  {}   Elves moved: {}".format(rounds,moved))
        stae = staen
        stae = contract_field(stae)
        stae = expand_field(stae, border_width=5)
        staen = move_elves(stae,inds)
        inds = np.roll(inds,-1)
        moved = (np.abs(staen - stae)).sum()/2.0
        rounds = rounds + 1
    return rounds

    

### Making a test section

Where I can test out some ideas based on what's going on.



In [9]:
desc = {}
desc[0] = ['..............',
'..............',
'.......#......',
'.....###.#....',
'...#...#.#....',
'....#...##....',
'...#.###......',
'...##.#.##....',
'....#..#......',
'..............',
'..............',
'..............',]

desc[1] = ['..............',
'.......#......',
'.....#...#....',
'...#..#.#.....',
'.......#..#...',
'....#.#.##....',
'..#..#.#......',
'..#.#.#.##....',
'..............',
'....#..#......',
'..............',
'..............']

desc[2] = ['..............',
'.......#......',
'....#.....#...',
'...#..#.#.....',
'.......#...#..',
'...#..#.#.....',
'.#...#.#.#....',
'..............',
'..#.#.#.##....',
'....#..#......',
'..............',
'..............']

desc[3] = ['..............',
'.......#......',
'.....#....#...',
'..#..#...#....',
'.......#...#..',
'...#..#.#.....',
'.#..#.....#...',
'.......##.....',
'..##.#....#...',
'...#..........',
'.......#......',
'..............']

desc[4] = ['..............',
'.......#......',
'......#....#..',
'..#...##......',
'...#.....#.#..',
'.........#....',
'.#...###..#...',
'..#......#....',
'....##....#...',
'....#.........',
'.......#......',
'..............']

desc[5] = ['.......#......',
'..............',
'..#..#.....#..',
'.........#....',
'......##...#..',
'.#.#.####.....',
'...........#..',
'....##..#.....',
'..#...........',
'..........#...',
'....#..#......',
'..............']

desc[10] = ['.......#......',
'...........#..',
'..#.#..#......',
'......#.......',
'...#.....#..#.',
'.#......##....',
'.....##.......',
'..#........#..',
'....#.#..#....',
'..............',
'....#..#..#...',
'..............']


In [10]:


test = {}
for k in desc.keys():
    test[k] = np.array( [ process_string(x) for x in desc[k]])
    test[k] = expand_field(test[k],border_width=3)

test[0].shape

(18, 20)

In [11]:

for k in test.keys():
    if k> 0:
        gg = run_field(test[0],num_iters = k)
        print(k, (gg-test[k]).max(),(gg-test[k]).min())




1 0.0 0.0
2 0.0 0.0
3 0.0 0.0
4 0.0 0.0
5 0.0 0.0
10 0.0 0.0


In [12]:
score_field(test[0])

110.0

## Part 1

Which is rather anticlimatic becuase withe all of the above machinery, this is one line to produce a submission number.  Well, plus the second and third cells above which load in the data.

In [13]:
score_field(field)

4208.0

## Part 2

Where we need to run score field a lot of times.  So we need to modify it slightly to make number of iterations a parameter, and add a print statement for number of elves moving, but otherwise should be straightforward.

Update:  Using score field for the actual data wasn't working well and so had to actually write one more function to accomplish the part 2 task.

In [14]:
score_field(test[0], num_iters=22, verbose = True)

Field shape is now (68, 70) with 22 elves
Running round 1
	 Elves moved is 11.0
Running round 2
	 Elves moved is 11.0
Running round 3
	 Elves moved is 13.0
Running round 4
	 Elves moved is 14.0
Running round 5
	 Elves moved is 19.0
Running round 6
	 Elves moved is 8.0
Running round 7
	 Elves moved is 10.0
Running round 8
	 Elves moved is 11.0
Running round 9
	 Elves moved is 6.0
Running round 10
	 Elves moved is 9.0
Running round 11
	 Elves moved is 7.0
Running round 12
	 Elves moved is 6.0
Running round 13
	 Elves moved is 5.0
Running round 14
	 Elves moved is 6.0
Running round 15
	 Elves moved is 6.0
Running round 16
	 Elves moved is 4.0
Running round 17
	 Elves moved is 2.0
Running round 18
	 Elves moved is 2.0
Running round 19
	 Elves moved is 2.0
Running round 20
	 Elves moved is 0.0
Running round 21
	 Elves moved is 0.0
Running round 22
	 Elves moved is 0.0


146.0

In [15]:
run_field_until_static(test[0],verbose = True)

Round:  1   Elves moved: 11.0
Round:  2   Elves moved: 11.0
Round:  3   Elves moved: 13.0
Round:  4   Elves moved: 14.0
Round:  5   Elves moved: 19.0
Round:  6   Elves moved: 8.0
Round:  7   Elves moved: 10.0
Round:  8   Elves moved: 11.0
Round:  9   Elves moved: 6.0
Round:  10   Elves moved: 9.0
Round:  11   Elves moved: 7.0
Round:  12   Elves moved: 6.0
Round:  13   Elves moved: 5.0
Round:  14   Elves moved: 6.0
Round:  15   Elves moved: 6.0
Round:  16   Elves moved: 4.0
Round:  17   Elves moved: 2.0
Round:  18   Elves moved: 2.0
Round:  19   Elves moved: 2.0


20

In [16]:
run_field_until_static(field, verbose = True)

Round:  1   Elves moved: 753.0
Round:  2   Elves moved: 626.0
Round:  3   Elves moved: 512.0
Round:  4   Elves moved: 489.0
Round:  5   Elves moved: 491.0
Round:  6   Elves moved: 501.0
Round:  7   Elves moved: 442.0
Round:  8   Elves moved: 453.0
Round:  9   Elves moved: 456.0
Round:  10   Elves moved: 486.0
Round:  11   Elves moved: 443.0
Round:  12   Elves moved: 477.0
Round:  13   Elves moved: 470.0
Round:  14   Elves moved: 490.0
Round:  15   Elves moved: 479.0
Round:  16   Elves moved: 502.0
Round:  17   Elves moved: 488.0
Round:  18   Elves moved: 526.0
Round:  19   Elves moved: 507.0
Round:  20   Elves moved: 564.0
Round:  21   Elves moved: 538.0
Round:  22   Elves moved: 552.0
Round:  23   Elves moved: 526.0
Round:  24   Elves moved: 559.0
Round:  25   Elves moved: 567.0
Round:  26   Elves moved: 625.0
Round:  27   Elves moved: 600.0
Round:  28   Elves moved: 624.0
Round:  29   Elves moved: 611.0
Round:  30   Elves moved: 651.0
Round:  31   Elves moved: 606.0
Round:  32   Elve

Round:  251   Elves moved: 1227.0
Round:  252   Elves moved: 1293.0
Round:  253   Elves moved: 1212.0
Round:  254   Elves moved: 1250.0
Round:  255   Elves moved: 1173.0
Round:  256   Elves moved: 1275.0
Round:  257   Elves moved: 1216.0
Round:  258   Elves moved: 1311.0
Round:  259   Elves moved: 1209.0
Round:  260   Elves moved: 1266.0
Round:  261   Elves moved: 1169.0
Round:  262   Elves moved: 1281.0
Round:  263   Elves moved: 1244.0
Round:  264   Elves moved: 1259.0
Round:  265   Elves moved: 1174.0
Round:  266   Elves moved: 1255.0
Round:  267   Elves moved: 1152.0
Round:  268   Elves moved: 1244.0
Round:  269   Elves moved: 1173.0
Round:  270   Elves moved: 1292.0
Round:  271   Elves moved: 1227.0
Round:  272   Elves moved: 1294.0
Round:  273   Elves moved: 1178.0
Round:  274   Elves moved: 1263.0
Round:  275   Elves moved: 1226.0
Round:  276   Elves moved: 1321.0
Round:  277   Elves moved: 1244.0
Round:  278   Elves moved: 1284.0
Round:  279   Elves moved: 1195.0
Round:  280   

Round:  495   Elves moved: 1106.0
Round:  496   Elves moved: 1145.0
Round:  497   Elves moved: 1107.0
Round:  498   Elves moved: 1145.0
Round:  499   Elves moved: 1126.0
Round:  500   Elves moved: 1182.0
Round:  501   Elves moved: 1069.0
Round:  502   Elves moved: 1133.0
Round:  503   Elves moved: 1107.0
Round:  504   Elves moved: 1159.0
Round:  505   Elves moved: 1117.0
Round:  506   Elves moved: 1179.0
Round:  507   Elves moved: 1109.0
Round:  508   Elves moved: 1174.0
Round:  509   Elves moved: 1146.0
Round:  510   Elves moved: 1190.0
Round:  511   Elves moved: 1116.0
Round:  512   Elves moved: 1169.0
Round:  513   Elves moved: 1073.0
Round:  514   Elves moved: 1163.0
Round:  515   Elves moved: 1141.0
Round:  516   Elves moved: 1154.0
Round:  517   Elves moved: 1098.0
Round:  518   Elves moved: 1176.0
Round:  519   Elves moved: 1084.0
Round:  520   Elves moved: 1142.0
Round:  521   Elves moved: 1103.0
Round:  522   Elves moved: 1148.0
Round:  523   Elves moved: 1056.0
Round:  524   

Round:  752   Elves moved: 487.0
Round:  753   Elves moved: 491.0
Round:  754   Elves moved: 475.0
Round:  755   Elves moved: 440.0
Round:  756   Elves moved: 445.0
Round:  757   Elves moved: 436.0
Round:  758   Elves moved: 444.0
Round:  759   Elves moved: 433.0
Round:  760   Elves moved: 437.0
Round:  761   Elves moved: 438.0
Round:  762   Elves moved: 447.0
Round:  763   Elves moved: 439.0
Round:  764   Elves moved: 450.0
Round:  765   Elves moved: 420.0
Round:  766   Elves moved: 434.0
Round:  767   Elves moved: 441.0
Round:  768   Elves moved: 423.0
Round:  769   Elves moved: 410.0
Round:  770   Elves moved: 382.0
Round:  771   Elves moved: 356.0
Round:  772   Elves moved: 367.0
Round:  773   Elves moved: 341.0
Round:  774   Elves moved: 349.0
Round:  775   Elves moved: 335.0
Round:  776   Elves moved: 330.0
Round:  777   Elves moved: 340.0
Round:  778   Elves moved: 356.0
Round:  779   Elves moved: 351.0
Round:  780   Elves moved: 330.0
Round:  781   Elves moved: 315.0
Round:  78

Round:  1007   Elves moved: 15.0
Round:  1008   Elves moved: 9.0
Round:  1009   Elves moved: 6.0
Round:  1010   Elves moved: 4.0
Round:  1011   Elves moved: 6.0
Round:  1012   Elves moved: 6.0
Round:  1013   Elves moved: 2.0
Round:  1014   Elves moved: 2.0
Round:  1015   Elves moved: 2.0


1016